In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = (
    SparkSession.builder
    .master("spark://MacBook-Pro.local:7077")
    .appName("test_mimic")
    #.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "../../jars/postgresql-42.2.19.jar")
    .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
    .getOrCreate()
)

# Read DB
df = spark.read.jdbc(
    DB_URI,
    "(SELECT row_id, subject_id, gender, dob FROM mimiciii.patients) as patients",
    properties={"user": USER, "password": PASSWORD, "driver": "org.postgresql.Driver"},
)

df.show()

+------+----------+------+-------------------+
|row_id|subject_id|gender|                dob|
+------+----------+------+-------------------+
|   234|       249|     F|2075-03-13 00:00:00|
|   235|       250|     F|2164-12-27 00:00:00|
|   236|       251|     M|2090-03-15 00:00:00|
|   237|       252|     M|2078-03-06 00:00:00|
|   238|       253|     F|2089-11-26 00:00:00|
|   239|       255|     M|2109-08-05 00:00:00|
|   240|       256|     M|2086-07-31 00:00:00|
|   241|       257|     F|2031-04-03 00:00:00|
|   242|       258|     F|2124-09-19 00:00:00|
|   243|       260|     F|2105-03-23 00:00:00|
|   244|       261|     M|2025-08-04 00:00:00|
|   245|       262|     M|2090-01-05 00:00:00|
|   246|       263|     M|2104-06-18 00:00:00|
|   247|       264|     F|2162-11-30 00:00:00|
|   248|       265|     M|2093-08-01 00:00:00|
|   249|       266|     F|2090-12-17 00:00:00|
|   250|       267|     F|2131-09-05 00:00:00|
|   251|       268|     F|2132-02-21 00:00:00|
|   252|     

In [4]:
# Fetch all the other useful

gender_df = df.withColumn(
    "fhir_gender",
    F.when(df["gender"] == "F", "female").when(df["gender"] == "M", "male").otherwise("unknown")
)
gender_df.show()

+------+----------+------+-------------------+-----------+
|row_id|subject_id|gender|                dob|fhir_gender|
+------+----------+------+-------------------+-----------+
|   234|       249|     F|2075-03-13 00:00:00|     female|
|   235|       250|     F|2164-12-27 00:00:00|     female|
|   236|       251|     M|2090-03-15 00:00:00|       male|
|   237|       252|     M|2078-03-06 00:00:00|       male|
|   238|       253|     F|2089-11-26 00:00:00|     female|
|   239|       255|     M|2109-08-05 00:00:00|       male|
|   240|       256|     M|2086-07-31 00:00:00|       male|
|   241|       257|     F|2031-04-03 00:00:00|     female|
|   242|       258|     F|2124-09-19 00:00:00|     female|
|   243|       260|     F|2105-03-23 00:00:00|     female|
|   244|       261|     M|2025-08-04 00:00:00|       male|
|   245|       262|     M|2090-01-05 00:00:00|       male|
|   246|       263|     M|2104-06-18 00:00:00|       male|
|   247|       264|     F|2162-11-30 00:00:00|     femal

# WRITE

In [5]:
gender_df.write.mode("overwrite").parquet("/tmp/spark-lake/big_mimic/patients")

# READ

In [6]:
pat_read = spark.read.parquet("/tmp/spark-lake/big_mimic/patients")
pat_read.show()

+------+----------+------+-------------------+-----------+
|row_id|subject_id|gender|                dob|fhir_gender|
+------+----------+------+-------------------+-----------+
|   234|       249|     F|2075-03-13 00:00:00|     female|
|   235|       250|     F|2164-12-27 00:00:00|     female|
|   236|       251|     M|2090-03-15 00:00:00|       male|
|   237|       252|     M|2078-03-06 00:00:00|       male|
|   238|       253|     F|2089-11-26 00:00:00|     female|
|   239|       255|     M|2109-08-05 00:00:00|       male|
|   240|       256|     M|2086-07-31 00:00:00|       male|
|   241|       257|     F|2031-04-03 00:00:00|     female|
|   242|       258|     F|2124-09-19 00:00:00|     female|
|   243|       260|     F|2105-03-23 00:00:00|     female|
|   244|       261|     M|2025-08-04 00:00:00|       male|
|   245|       262|     M|2090-01-05 00:00:00|       male|
|   246|       263|     M|2104-06-18 00:00:00|       male|
|   247|       264|     F|2162-11-30 00:00:00|     femal